In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_LR.csv', index_col=0)
df.head(10)

# **Ustunlar ta'rifi**
* ***district*** - uy joylashgan tuman
* ***rooms*** - xonalar soni
* ***size*** - uy maydoni (kv.m)
* ***level*** - uy
* ***max_levels*** - uylaring qavati

In [30]:
df['district'].value_counts()

# Chilonzor tumanidagi uylarning narxini bashorat qilib ko'ramiz

In [31]:
housing = df[df.district=='Чиланзарский']
housing.head()

In [32]:
X = housing['size'].to_numpy()
X

In [33]:
y = housing['price'].to_numpy()
y

# Narx va uy maydoni o'rtasida chiziqli bog'liqlik bor ekanini tekshirib ko'ramiz.

In [34]:
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=housing, x='size', y='price')
plt.show()

In [36]:
plt.figure(figsize=(10,6))
sns.regplot(data=housing, x='size', y='price', line_kws={"color": "red"})
plt.show()

![](https://i.imgur.com/zZoXZxI.png)

**Bu yerda ~x - X ustun uchun o'rtracha qiymat, ~y - y ustun uchun o'rtracha qiymat**

In [37]:
Xmean = np.mean(X)
ymean = np.mean(y)

In [40]:
theta1 = sum((X-Xmean)*(y-ymean))/sum((X-Xmean)**2)
print(f"theta1={theta1}")

In [41]:
theta0 = ymean - theta1*Xmean
print(f"theta0={theta0}")

![](https://i.imgur.com/iT5tffj.png)

In [46]:
x_test = housing.sample(10, random_state=42)['size'].to_numpy()
print(f"x_test={x_test}")
y_test = housing.sample(10, random_state=42)['price'].to_numpy()
print(f"y_test={y_test}")

In [47]:
y_predict = theta0 + theta1*x_test
print(f"y_predict={y_predict}")

![](https://i.imgur.com/GJH9CGy.png)
![](https://i.imgur.com/aXL9iWa.png)

In [49]:
# MAE
MAE = np.sum(np.absolute(y_predict-y_test))/len(y_test)
print(f"MAE={MAE}")

**MAE=4140.35031634715**

In [50]:
# RMSE
RMSE = np.sqrt(np.sum((y_predict-y_test)**2)/len(y_test))
print(f"RMSE={RMSE}")

**RMSE=5882.051673173715**

# Scikit-learn yordamida LR

**Keling endi yuqoridagi amallarni scikit-learn yordamida bajaramiz.**

**Avvalo ma'lumotlarni train va testga bo'lib olamiz.**

In [51]:
housing.shape

In [56]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.10, random_state=42)

In [57]:
from sklearn import linear_model
LR_model = linear_model.LinearRegression()

x_train = np.asanyarray(train_set[['size']])
y_train = np.asanyarray(train_set[['price']])

# Modelni tarbiyalaymiz (train)
LR_model.fit(x_train, y_train)

#theta1 va theta0 koeefisentlarni ajratib olamiz 
theta1 = LR_model.coef_[0][0]
theta0 = LR_model.intercept_[0]
print (f'theta1={theta1}')
print (f'theta0={theta0}')

In [58]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=housing, x='size', y='price')
plt.plot(x_train, theta1*x_train + theta0, '-r')
plt.xlabel("Size")
plt.ylabel("Price")

# Modelni baholaymiz

In [60]:
x_test = np.asanyarray(test_set[['size']])
y_test = np.asanyarray(test_set[['price']])

In [62]:
y_predict = LR_model.predict(x_test)

In [63]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

MAE = mean_absolute_error(y_test, y_predict)
RMSE = np.sqrt(mean_squared_error(y_test, y_predict))
print(f"MAE={MAE}")
print(f"RMSE={RMSE}")